<a href="https://colab.research.google.com/github/nosugarfor533/https-github.com-xinyu-wu-BMDW-Assignment/blob/master/Copy_of_Deseq2_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
?system
version

               _                           
platform       x86_64-pc-linux-gnu         
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          3                           
minor          6.3                         
year           2020                        
month          02                          
day            29                          
svn rev        77875                       
language       R                           
version.string R version 3.6.3 (2020-02-29)
nickname       Holding the Windsock        



---

Analyze gene count data using Deseq2 

---



In [ ]:
install.packages("rgl", repos = "http://cran.rstudio.com/")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘fastmap’, ‘miniUI’, ‘webshot’, ‘shiny’, ‘manipulateWidget’




In [ ]:
# Install specific version of XML for compatibility with following packages (CosRank and Deseq2)
packageurl <- "https://cran.r-project.org/src/contrib/Archive/XML/XML_3.98-1.20.tar.gz"
install.packages(packageurl, repos=NULL, type="source")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages("ConsRank", repos = "http://cran.rstudio.com/")
library("ConsRank")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘data.table’, ‘rlist’, ‘proxy’, ‘gtools’


Loading required package: rgl

Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”

Attaching package: ‘ConsRank’


The following object is masked from ‘package:base’:

    labels




In [ ]:
system("add-apt-repository -y ppa:marutter/rrutter")
system("add-apt-repository -y ppa:marutter/c2d4u")
system("apt-get update")
system("apt install -y r-cran-rstan")

In [ ]:
install.packages("ggplot2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
BiocManager::install(version="3.9", ask=FALSE)

Bioconductor version 3.9 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

Installing package(s) 'BiocVersion'

Installing package(s) 'curl', 'dplyr', 'farver', 'fs', 'rex', 'rprojroot',
  'tidyr'

also installing the dependency ‘cpp11’




In [ ]:
BiocManager::install("DESeq2", version="3.9")

In [ ]:
# Download the Data

In [ ]:
# countsName <- "https://bioconnector.github.io/workshops/data/airway_scaledcounts.csv"
# download.file(countsName, destfile = "airway_scaledcounts.csv", method = "auto")

# countData <- read.csv('airway_scaledcounts.csv', header = TRUE, sep = ",")
# head(countData)

In [ ]:
# metaDataName <- "https://bioconnector.github.io/workshops/data/airway_metadata.csv"
# download.file(metaDataName, destfile = "airway_metadata.csv", method = "auto")
# metaData <- read.csv('airway_metadata.csv', header = TRUE, sep = ",")
# metaData


# metadata matrix: an matrix in the form of pandas dataframe, 
# see DESeq2 manual, samplenames as rownames following treatment type (A,B)
#     sampleA1        A
#     sampleA2        A
#     sampleB1        B
#     sampleB2        B

In [ ]:
countData <- read.csv('gene_matrix.csv', header = TRUE, sep = ",",row.names=NULL)
head(countData)

In [ ]:
metaData <- read.csv('meta_data.csv', header = TRUE, sep = ",",row.names=NULL)
metaData

**Differential analysis Calculation with Salmon data using DESEQ2**

Construct DESEQDataSet Object


In [ ]:
library(ggplot2)
library( "DESeq2" )

In [ ]:
dds <- DESeqDataSetFromMatrix(countData=countData, 
                              colData=metaData, 
                              design=~status, tidy = TRUE)

In [ ]:
#Design specifies how the counts from each gene depend on our variables in the metadata
#For this dataset the factor we care about is our treatment status (status)
#tidy=TRUE argument, which tells DESeq2 to output the results table with rownames as a first #column called 'row.

#let's see what this object looks like
dds

Run DESEQ function

In [ ]:
dds <- DESeq(dds)
#estimateSizeFactors
#This calculates the relative library depth of each sample 

#estimateDispersions
#estimates the dispersion of counts for each gene 

#nbinomWaldTest
#calculates the significance of coefficients in a Negative Binomial GLM using the size and dispersion outputs

Results Table

In [ ]:
res <- results(dds, tidy=TRUE)
head(res)

Summary of differential gene expression

In [ ]:
summary(res)

Sort summary list by p-value

In [ ]:
res <- res[order(res$pvalue),]
head(res)


Plot Counts

In [ ]:
#we can use plotCounts fxn to compare the normalized counts
#between treated and control groups for our top 6 genes
par(mfrow=c(2,3))

plotCounts(dds, gene="ENSG00000251638", intgroup="status")
plotCounts(dds, gene="ENSG00000166295", intgroup="status")
plotCounts(dds, gene="ENSG00000270570", intgroup="status")
plotCounts(dds, gene="ENSG00000092820", intgroup="status")
plotCounts(dds, gene="ENSG00000139626", intgroup="status")
plotCounts(dds, gene="ENSG00000258597", intgroup="status")
#Next steps in exploring these data...BLAST to database to find associated gene function

Volcano Plot

In [ ]:
library(repr)
options(repr.plot.width=10, repr.plot.height=7) 

In [ ]:
#reset par
par(mfrow=c(1,1))
# Make a basic volcano plot
with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main="Volcano plot", xlim=c(-3,3)))

# Add colored points: blue if padj<0.01, red if log2FC>1 and padj<0.05)
with(subset(res, padj<.01 ), points(log2FoldChange, -log10(pvalue), pch=20, col="blue"))
with(subset(res, padj<.01 & abs(log2FoldChange)>2), points(log2FoldChange, -log10(pvalue), pch=20, col="red"))


In [ ]:
#First we need to transform the raw count data
#vst function will perform variance stabilizing transformation

vsdata <- vst(dds, blind=FALSE)

In [ ]:
plotPCA(vsdata, intgroup="status")
#using the DESEQ2 plotPCA fxn we can look at how our samples group by treatment
